<a href="https://colab.research.google.com/github/SeeBook2022/SeeBook-OCR/blob/main/03.ocr/test_img_preprocessing/ocr_img_preprocessing_test_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 설정


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install --upgrade google-api-python-client

In [ ]:
%pip install google-cloud-vision

In [ ]:
import os
import pandas as pd

In [ ]:
!ls -l $GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  '/content/drive/MyDrive/aiffel_hackathon3/google_api_key/my_google_api_key.json'

total 8
drwx------ 6 root root 4096 Feb 28 05:50 drive
drwxr-xr-x 1 root root 4096 Feb 18 14:33 sample_data


### 새로운 데이터 파일명 수정

In [ ]:
def change_name(file_path, file_list):
    for idx, filename in enumerate(file_list):
        os.rename(file_path + filename, file_path + 'test_title' + str(idx+1) +'.jpg')

In [ ]:
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/test_dataset3_ver3/'
file_list = os.listdir(img_dir)
file_list.sort(key = lambda x : int(x[10:-4]))

In [ ]:
change_name(img_dir, file_list)

### 텍스트 처리, csv 파일 준비


In [ ]:
# 후처리 함수 (특수문자 및 공백 제거)
def preprocess_sentence(sentence):
    import re
    sentence = re.sub('[-=+,「」#/\?:^$.@*\"※~&%ㆍ•!『』\\‘|\(\)\[\]\<\>`\'…》]','', sentence)
    sentence = sentence.replace(' ', '') # 공백 제거
    # sentence = sentence.strip()
    return sentence

In [ ]:
test_sentence = '안녕하 세요. 반갑습~니다!0101234a?b,長'
print(preprocess_sentence(test_sentence))

안녕하세요반갑습니다0101234ab長


In [ ]:
import math
import time
start = time.time()
test_sentence = '안녕하 세요. 반갑습~니다!0101234a?b,長'
preprocess_sentence(test_sentence)
end = time.time()
print(f"{end - start:.5f} sec")

0.00008 sec


In [ ]:
# 정량 평가를 위한 테스트 데이터 라벨 csv 파일 불러오기
compare_df = pd.read_csv('/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/test_dataset3_ver3.csv', encoding = 'UTF-8')

In [ ]:
for idx, sentence in enumerate(compare_df['책제목']):
    compare_df['책제목'][idx] = preprocess_sentence(sentence)

In [ ]:
# 정량 평가를 위한 테스트 데이터 라벨 csv 파일 불러오기2
compare_df2 = pd.read_csv('/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/ocr_test_ver3_result2.csv', encoding = 'UTF-8')

## google vision api (baseline)

In [ ]:
def detect_text(path): 
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    result = ''
    for idx, text in enumerate(texts):
        if idx > 0: result += text.description 
    return result

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/test_dataset3_ver3/'
file_list = os.listdir(img_dir)
file_list.sort(key = lambda x : int(x[10:-4]))

In [ ]:
# 시간 측정(10장 기준+텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text(paths)))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

7.49846 sec
['세월이가면', '님의침묵', '먼후일', '아', '모모', '친국의열쇠', '고난이선물이다', '오뒷세이아', '25시', '활']


In [ ]:
# 시간 측정(10장 기준+텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
detect_text(paths)
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

0.37260 sec
['세월이가면', '님의침묵', '먼후일', '아', '모모', '친국의열쇠', '고난이선물이다', '오뒷세이아', '25시', '활']


In [ ]:
result_list = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list.append(preprocess_sentence(detect_text(paths)))

## 이미지 처리 (ver.성연)

### 이미지 처리

In [ ]:
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/test_dataset3_ver3/'
file_list = os.listdir(img_dir)
file_list.sort(key = lambda x : int(x[10:-4]))

In [ ]:
def detect_text2(img_bytes):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    image = vision.Image(content=img_bytes) # byte로 받아서 바로 실행

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    result = ''
    for idx, text in enumerate(texts):
        if idx > 0: result += text.description 
    return result

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
        
def img2gray3(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백 처리
    img_resize = cv2.resize(img_gray, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_AREA) # resize(영역 보간)
    add_dst = cv2.add(img_resize, 30) # 밝기 조절

    img_bytes = cv2.imencode('.jpg', add_dst)[1].tobytes()

    return img_bytes

In [ ]:
def img2gray(path):
    import cv2
    img = cv2.imread(path) # 이미지 불러오기
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백 처리
    img_resize = cv2.resize(img_gray, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_AREA) # resize(영역 보간)
    add_dst = cv2.add(img_resize, 30) # 밝기 조절

    # img_bytes = base64.b64encode(cv2.imencode('.jpg', img)[1]).decode()
    img_bytes = cv2.imencode('.jpg', add_dst)[1].tobytes()

    return img_bytes

In [ ]:
import cv2
img = cv2.imread(img_dir + file_list[0])

In [ ]:
import math
import time
start = time.time()
img2gray(img_dir+file_list[0])
end = time.time()
print(f"{end - start:.5f} sec")

0.01299 sec


In [ ]:
import math
import time
start = time.time()
preprocess_sentence(detect_text2(img2gray3(img)))
end = time.time()
print(f"{end - start:.5f} sec")

0.68670 sec


In [ ]:
import math
import time
start = time.time()
preprocess_sentence(detect_text2(img2gray(img_dir+file_list[0])))
end = time.time()
print(f"{end - start:.5f} sec")

0.66059 sec


In [ ]:
def img2gray2(path):
    import cv2
    img = cv2.imread(path) # 이미지 불러오기
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백 처리
    img_resize = cv2.resize(img_gray, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_CUBIC) # resize(바이큐빅 보간)
    add_dst = cv2.add(img_resize, 30) # 밝기 조절

    img_bytes = cv2.imencode('.jpg', add_dst)[1].tobytes()

    return img_bytes

### OCR test

In [ ]:
def detect_text2(img_bytes):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    image = vision.Image(content=img_bytes) # byte로 받아서 바로 실행

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    result = ''
    for idx, text in enumerate(texts):
        if idx > 0: result += text.description 
    return result

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/test_dataset3_ver3/'
file_list = os.listdir(img_dir)
file_list.sort(key = lambda x : int(x[10:-4]))

In [ ]:
# 시간 측정(10장 기준+ 이미지 처리 + 텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text2(img2gray(paths))))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

7.37846 sec
['세월이가면', '님의침묵', '먼후일', '일리아스', '모모', '천국의열쇠', '고난이선물이다', '오뒷세이아', '25시', '부활']


In [ ]:
result_list2 = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list2.append(preprocess_sentence(detect_text2(img2gray(paths))))

In [ ]:
# 시간 측정(10장 기준+ 이미지 처리2 + 텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text2(img2gray2(paths))))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

6.58873 sec
['세월이가면', '님의침묵', '먼후일', '아일리。', '모모', '천국의열쇠', '고난이선물이다', '오뒷세이', '25시', '부활']


In [ ]:
result_list3 = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list3.append(preprocess_sentence(detect_text2(img2gray2(paths))))

## 이미지 처리 (ver.유진)

### 이미지 처리

In [ ]:
import cv2
import os
import pandas as pd
import time
import re
import datetime
import io
import numpy as np

In [ ]:
def img_Contrast(path): 
    img = cv2.imread(path)
    # -----Converting image to LAB Color model----------------------------------- 
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB) 
    
    # -----Splitting the LAB image to different channels------------------------- 
    l, a, b = cv2.split(lab) 
    # -----Applying CLAHE to L-channel------------------------------------------- 
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)) 
    cl = clahe.apply(l) 
    # -----Merge the CLAHE enhanced L-channel with the a and b channel----------- 
    limg = cv2.merge((cl, a, b)) 
    # # -----Converting image from LAB Color model to RGB model-------------------- 
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR) 
    img_bytes = cv2.imencode('.jpg', final)[1].tobytes()
    return img_bytes

### OCR test

In [ ]:
def detect_text2(img_bytes):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    image = vision.Image(content=img_bytes) # byte로 받아서 바로 실행

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    result = ''
    for idx, text in enumerate(texts):
        if idx > 0: result += text.description 
    return result

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/test_dataset3_ver3/'
file_list = os.listdir(img_dir)
file_list.sort(key = lambda x : int(x[10:-4]))

In [ ]:
# 시간 측정(10장 기준+ 이미지 처리 + 텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text2(img_Contrast(paths))))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

9.46714 sec
['세월이가면', '님의침묵', '먼후일', '', '豆', '전국의열쇠', '고난이선물이다', '오뒷세이아', '시', '']


In [ ]:
result_list4 = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list4.append(preprocess_sentence(detect_text2(img_Contrast(paths))))

## 이미지 처리 (ver.진우)

### 이미지 처리

In [ ]:
import cv2
import os
import pandas as pd
import time
import re
import datetime
import io
import numpy as np

In [ ]:
# resize(1.5배) 
def resize_img(path):
    img = cv2.cvtColor(cv2.imread(path), cv2.IMREAD_COLOR)
    resize_img = cv2.resize(img, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_AREA)
    img_bytes = cv2.imencode('.jpg', resize_img)[1].tobytes()
    return img_bytes

In [ ]:
# grayscale + contrast
def gray_contrast(path):
    gray_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) # grayscale 적용
    gray_cons = np.clip((1 + 1.0) * gray_img - 128 * 1.0, 0, 255).astype(np.uint8) # contrast 적용
    img_bytes = cv2.imencode('.jpg', gray_cons)[1].tobytes()
    return img_bytes

In [ ]:
# grayscale + contrast + resize
def gray_contrast_resize(path):
    gray_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    resize_img = cv2.resize(gray_img, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_AREA)
    gray_resize_cons = np.clip((1 + 1.0) * resize_img - 128 * 1.0, 0, 255).astype(np.uint8)
    img_bytes = cv2.imencode('.jpg', gray_resize_cons)[1].tobytes()
    return img_bytes

In [ ]:
# 시간 측정(10장 기준+ 이미지 처리 + 텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text2(resize_img(paths))))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

5.85151 sec
['세월이가면', '님의침묵', '일먼후', '아일리。', '모모', '천국의열쇠', '고난이선물이다', '오뒷세이야', '25시', '부활']


In [ ]:
result_list5 = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list5.append(preprocess_sentence(detect_text2(resize_img(paths))))

In [ ]:
# 시간 측정(10장 기준+ 이미지 처리 + 텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text2(gray_contrast(paths))))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

5.25584 sec
['세월이가면', '님의침묵', '먼후일', '아', '모모', '천국의열쇠', '고난이선물이다', '오뒷세이아', '25', '부활']


In [ ]:
result_list6 = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list6.append(preprocess_sentence(detect_text2(gray_contrast(paths))))

In [ ]:
# 시간 측정(10장 기준+ 이미지 처리 + 텍스트 처리 + 리스트 저장)
import math
import time
start = time.time()
test = []
for i in range(10):
    paths = img_dir + file_list[i]
    test.append(preprocess_sentence(detect_text2(gray_contrast_resize(paths))))
end = time.time()
print(f"{end - start:.5f} sec")
print(test)

5.97140 sec
['세월이가면', '님의침묵', '먼후일', '일리아스', '모', '천국의열쇠', '고난이선물이다', '아오뒷세', '시25', '']


In [ ]:
result_list7 = []
for i in range(380):
    paths = img_dir + file_list[i]
    result_list7.append(preprocess_sentence(detect_text2(gray_contrast_resize(paths))))

## 검증

### 1차 (baseline, ver 성연)

In [ ]:
compare_df = compare_df.assign(result1 = result_list, result2 = result_list2, result3 = result_list3)

In [ ]:
# 라벨, 예측값 비교 (성능 검증) 함수
def compare_result(df, col, num):
    from difflib import SequenceMatcher
    ratio_list = []
    for i in range(num):
        if df[col][i] == '':
            ratio_list.append('0')
        elif df['책제목'][i] == df[col][i]:
            ratio_list.append('100')
        else:
            str1 = df.iloc[i]['책제목']
            str2 = df.iloc[i][col]
            ratio = SequenceMatcher(None, str1, str2).ratio()
            ratio_list.append(round(ratio*100))
    return ratio_list

In [ ]:
com1 = compare_result(compare_df, 'result1', 380)
com2 = compare_result(compare_df, 'result2', 380)
com3 = compare_result(compare_df, 'result3', 380)

In [ ]:
compare_df = compare_df.assign(compare1 = com1, compare2 = com2, compare3 = com3)

In [ ]:
compare_df

,파일명,책제목,result1,result2,result3,compare1,compare2,compare3
0,test_title1.jpg,세월이가면,세월이가면,세월이가면,세월이가면,100,100,100
1,test_title2.jpg,님의침묵,님의침묵,님의침묵,님의침묵,100,100,100
2,test_title3.jpg,먼후일,먼후일,먼후일,먼후일,100,100,100
3,test_title4.jpg,일리아스,아,일리아스,아일리。,40,100,50
4,test_title5.jpg,모모,모모,모모,모모,100,100,100
...,...,...,...,...,...,...,...,...
375,test_title376.jpg,파리지앵마카롱,파리지앵마카롱,파리지앵마카롱7,파리지앵마카롱,100,93,100
376,test_title377.jpg,아이러브와플,,,,0,0,0
377,test_title378.jpg,280일태교다이어리헬로마이베이비,280일태교다이어리헬로마이베이비,280일태교다이어리헬로마이베이비,280일태교다이어리헬로마이베이비,100,100,100
378,test_title379.jpg,홍차의비밀,홍차의비밀,홍차의비밀,홍차의비밀,100,100,100


In [ ]:
compare_df.to_csv('/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/ocr_test_ver3_result2.csv', index=False)

In [ ]:
print(compare_df['compare1'].value_counts(ascending = False).head(10))
print(compare_df['compare2'].value_counts(ascending = False).head(10))
print(compare_df['compare3'].value_counts(ascending = False).head(10))

100    282
0       11
67       9
80       7
86       7
50       6
83       5
92       4
88       4
89       4
Name: compare1, dtype: int64
100    299
67       8
0        7
0        6
86       6
91       5
50       4
93       4
58       3
80       3
Name: compare2, dtype: int64
100    290
86       7
0        7
83       6
0        5
67       5
50       4
91       4
95       4
92       4
Name: compare3, dtype: int64


### 2차 (ver유진, 진우)

In [ ]:
compare_df2 = compare_df2.assign(result4 = result_list4, result5 = result_list5, result6 = result_list6, result7 = result_list7)

In [ ]:
com4 = compare_result(compare_df2, 'result4', 380)
com5 = compare_result(compare_df2, 'result5', 380)
com6 = compare_result(compare_df2, 'result6', 380)
com7 = compare_result(compare_df2, 'result7', 380)

In [ ]:
compare_df2 = compare_df2.assign(compare4 = com4, compare5 = com5, compare6 = com6, compare7 = com7)

In [ ]:
compare_df2.to_csv('/content/drive/MyDrive/aiffel_hackathon3/OCR_test_ver3/ocr_test_ver3_result3.csv', index=False)

In [ ]:
print(compare_df2['compare4'].value_counts(ascending = False).head(10))
print(compare_df2['compare5'].value_counts(ascending = False).head(10))
print(compare_df2['compare6'].value_counts(ascending = False).head(10))
print(compare_df2['compare7'].value_counts(ascending = False).head(10))

100    257
0       19
0       11
80       9
89       8
88       6
83       5
86       5
92       4
50       4
Name: compare4, dtype: int64
100    287
0        8
0        6
80       5
88       5
91       5
67       5
50       4
12       3
83       3
Name: compare5, dtype: int64
100    269
0       11
89       8
0        8
86       8
80       7
50       6
90       5
67       5
56       4
Name: compare6, dtype: int64
100    279
0       12
0       10
67      10
86       8
80       6
95       4
88       4
90       4
91       4
Name: compare7, dtype: int64
